# Overview of the mainscripts to run the free energy exploration
*Cyril Rommens, s12495719, masterproject MSc Physics and Astronomy: Computational Physics of Complex Systems*

**Content**
1.	raw_to_PR: 	Timeseries -> PR_timeseries (Is on harddrive)
2.	Data_to_CC: 	Timeseries + PR_timeseries -? pruned_clique_complex + entropies/mutinfo/F’s
3.	CC_to_F: 	Pruned_clique_complex + L_p -> Knill’s F (for this L_p needs to be generated before the function/loop. Matrix decompression needs to be used to select which L_p columns/rows)
4.	CC_to_minF_scipy: 	Pruned_clique_complex -> Knill’s min F from scipy P
5.	CC_to_minF_custom:	Pruned_clique_complex -> Knill’s min F from custom P

**1 raw_to_PR: 	Timeseries -> PR_timeseries**

In [ ]:
#Put PR code here

**2.	Data_to_CC: 	Timeseries + PR_timeseries -> pruned_clique_complex + entropies/mutinfo/F’s**

In [28]:
def obtain_mutual_information(filename, max_d, number_of_variables): 

    # Import time series data
    df = pd.read_csv(filename, sep='\t', header=None)

    # Mask using the 99th and 1th percentile
    stacked_series = df.stack()
    quantile_99 = stacked_series.quantile(0.99)
    quantile_01 = stacked_series.quantile(0.01)
    df[df > quantile_99] = quantile_99
    df[df < quantile_01] = quantile_01

    # Initialize an empty DataFrame to hold discretized values
    discretized_time_series = pd.DataFrame()
    max_BOLD = df.max().max()
    min_BOLD = df.min().min()
    desired_number_of_bins = 16
    stepsize = (max_BOLD-min_BOLD)/desired_number_of_bins
    bin_edges = np.arange(min_BOLD, max_BOLD, stepsize)

    # Iterate over each column of 'df'
    for col in df.columns:
        data = df[col].tolist()
        bin_numbers = np.digitize(data, bin_edges)
        
        # Create a DataFrame with the current column's discretized values
        col_df = pd.DataFrame({col: bin_numbers}, index=df.index)
        
        # Concatenate the new DataFrame along the columns axis
        discretized_time_series = pd.concat([discretized_time_series, col_df], axis=1)

    # Import the information topology functions needed
    %run infotopo.py
    
    # Settings to use infotopo functions
    dataset = np.array(discretized_time_series).T
    work_on_transpose = False 
    nb_of_values = 16
    deformed_probability_mode = False
    supervised_mode = False
    forward_computation_mode = True
    sampling_mode = 1

    # Call infotopo functions for entropy, mutual information and free energy
    information_topo = infotopo(dimension_max = max_d, 
                                dimension_tot = number_of_variables, 
                                sample_size = 2400, 
                                work_on_transpose = work_on_transpose,
                                nb_of_values = nb_of_values, 
                                sampling_mode = sampling_mode, 
                                deformed_probability_mode = deformed_probability_mode,
                                supervised_mode = supervised_mode, 
                                forward_computation_mode = forward_computation_mode)

    Nentropie = information_topo.simplicial_entropies_decomposition(dataset) 
    Ninfomut = information_topo.simplicial_infomut_decomposition(Nentropie)
    Nfree_energy = information_topo.total_correlation_simplicial_lanscape(Nentropie)

    return Nentropie, Ninfomut, Nfree_energy

In [ ]:
# Run for max_d=3 and nb_variables=60 takes about 4min
%time Nentropie, Ninfomut, Nfree_energy = obtain_mutual_information('Cyril\\100206_rfMRI_REST1_Atlas_MSMAll_hp2000_clean_Schaefer2018_100Parcels_7Networks_Tian_Subcortex_S1_3T.txt', 3, 10)

In [ ]:
average_free_energy = sum(Nfree_energy.values())/len(Nfree_energy.values())
print(f'The average free energy of this subject is {average_free_energy}')

average_free_energy_component = sum(Ninfomut.values())/len(Ninfomut.values())
print(f'The free energy component (average mutual information) of this subject is {average_free_energy_component}')

In [44]:
print(Ninfomut)
clique_complex = [frozenset(key) for key in Ninfomut if key in Ninfomut]
print(clique_complex)

{(1,): 2.3158932632922564, (2,): 2.6034634969390407, (3,): 2.2046295200496218, (4,): 2.1044846023411123, (5,): 3.0029452453336924, (6,): 2.638108973868445, (7,): 2.3989199097781473, (8,): 2.615714896072481, (9,): 2.263040738115215, (10,): 2.6811687257127006, (1, 2): 0.03881460164945061, (1, 3): 0.03208233536630356, (1, 4): 0.018709167611643096, (1, 5): 0.025890859202976735, (1, 6): 0.025290038216225774, (1, 7): 0.021381681022933918, (1, 8): 0.02069406727950529, (1, 9): 0.11753803484330305, (1, 10): 0.04036144115299489, (2, 3): 0.023696574549664895, (2, 4): 0.025942486088583294, (2, 5): 0.03453746108139466, (2, 6): 0.026589932228832325, (2, 7): 0.024578331699744993, (2, 8): 0.027395668574090593, (2, 9): 0.029919078015017142, (2, 10): 0.05743714442760517, (3, 4): 0.034541888731590475, (3, 5): 0.029368583583354813, (3, 6): 0.01869813165726608, (3, 7): 0.024930450490373346, (3, 8): 0.037263893003677495, (3, 9): 0.02045462927904662, (3, 10): 0.023421576139748268, (4, 5): 0.04398813059588047

**3. CC_to_F: 	Pruned_clique_complex + L_p -> Knill’s F**

In [35]:
def count_occurrences(dicts):
    occurrences = {}
    # Iterate over all dictionaries
    for d in dicts:
        for key in d:
            occurrences[key] = occurrences.get(key, 0) + 1
    return occurrences

def generate_clique_probabilities(dict_list):  
      
    # Count occurrences
    occurrences = count_occurrences(dict_list)

    # Divide each value by the sum
    normalized_occurrences = {key: value /sum(occurrences.values()) for key, value in occurrences.items()}
    
    dict_values_list = []

    for i in range(0, len(dict_list)):
        # Initialize an empty list to store values
        values_list = []

        # Iterate over keys of dict1
        for key in dict_list[i]:
            # Check if the key exists in normalized_occurrences
            if key in normalized_occurrences:
                # If the key exists, append its corresponding value to the list
                values_list.append(normalized_occurrences[key])

        dict_values_list.append(np.array(values_list))

    return dict_values_list

In [42]:
%run Week_18_functions.py

In [46]:
# Test for some example clique_complexes
clique_complex_1 = {(1,):0.245, (2,):0.456, (1, 2): 0.123}
clique_complex_2 = {(2,):0.767, (3,):0.178, (2, 3): 0.367}
clique_complex_3 = {(1,):0.267, (4,):0.278, (6,): 0.128, (1, 4, 6):0.236}
clique_complex_1 = [frozenset(key) for key in clique_complex_1 if key in clique_complex_1]
clique_complex_2 = [frozenset(key) for key in clique_complex_2 if key in clique_complex_2]
clique_complex_3 = [frozenset(key) for key in clique_complex_3 if key in clique_complex_3]
clique_complex_list = [clique_complex_1, clique_complex_2, clique_complex_3]

# Generate Free energies
F_list = []
clique_probabilities_list = generate_clique_probabilities(clique_complex_list)

for i in range(0, len(clique_complex_list)):
    clique_complex = clique_complex_list[i]
    probability = clique_probabilities_list[i]
    matrix, inverse_connectivity_matrix = generate_inverse_connectivity_matrix(clique_complex)
    F = energy_function(probability, inverse_connectivity_matrix)
    F_list.append(F)

print(F_list)

[-0.010000000000000002, 0.010000000000000002, 0.015000000000000003]


**4. CC_to_minF_scipy: 	Pruned_clique_complex -> Knill’s min F from scipy P**

In [48]:
clique_complex_list

[[frozenset({1}), frozenset({2}), frozenset({1, 2})],
 [frozenset({2}), frozenset({3}), frozenset({2, 3})],
 [frozenset({1}), frozenset({4}), frozenset({6}), frozenset({1, 4, 6})]]

In [69]:
from scipy.optimize import minimize

def complete_f_generator_scipy(clique_complex):
    Q = generate_inverse_connectivity_matrix(clique_complex)[1]

    # Optimization settings
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = [(1e-10, None) for _ in range(len(Q))]
    x0 = np.full(len(Q), 1/len(Q))  # Initial guess

    # Store the latest optimized x0 and all free energies during minimization
    latest_x0 = None
    t = 1
    all_values = []

    # Callback function to collect values during minimization
    def callback(x):
        all_values.append(free_energy_function(x, Q, t))

    #for t in t_values:
    #    result = minimize(objective, x0, args=(Q, t), method='SLSQP', constraints=cons, bounds=bounds)
    #    minimized_values.append(result.fun)
    #    latest_x0 = result.x  # Update the latest optimized x0

    result = minimize(free_energy_function, x0, args=(Q, t), method='SLSQP', constraints=cons, bounds=bounds, callback=callback)
    return [result.fun, result.x]

In [71]:
print('Using scipy the minimum free energy and the corresponding clique probabilities are', complete_f_generator_scipy(clique_complex_list[0]))

Using scipy the minimum free energy and the corresponding clique probabilities are [-0.4999999996999998, array([5.00000000e-01, 5.00000000e-01, 1.00000064e-10])]


F minmisation using custom P

In [54]:
def complete_f_generator(clique_complex):
    # Generate connection matrix and inverse
    matrix, inverse_connectivity_matrix = generate_inverse_connectivity_matrix(clique_complex)

    # Generate maximum shannon entropy from uniform distribution
    n = len(inverse_connectivity_matrix)
    p_Smax = np.ones(n) / n
    max_entropy_value = shannon_entropy(p_Smax)

    # Generate minimum internal energy from analytical solution
    min_energy_probabilities = (np.inner(matrix,[1]*len(matrix)))/np.sum(matrix)
    min_energy_value = energy_function(min_energy_probabilities, inverse_connectivity_matrix)

    temperature = 0.5 # ratio between influence of internal energy and entropy
    iterations = 1000 # number of iterations to minimise F
    free_energy_history, f_probabilities = free_energy_minimisation(clique_complex, inverse_connectivity_matrix, iterations, temperature)

    ''' IF TEMPERATURE VARIATION
    # Generate minimum free energy by custom optimisation
    f_per_T = []
    p_per_T = []
    temperature_list = [0.5] #One temperature for now, to look at a range use here: np.arange(0, 1, 1)

    for temperature in temperature_list:
        free_energy_history, f_probabilities = free_energy_minimisation(clique_complex, inverse_connectivity_matrix, 1000, temperature)
        f_per_T.append(free_energy_history[-1])
        p_per_T.append(f_probabilities)

    return [max_entropy_value, min_energy_value, f_per_T, p_per_T, temperature_list]
    '''
    return [max_entropy_value, min_energy_value, free_energy_history[-1], f_probabilities]

In [68]:
complete_f_generator(clique_complex_list[0])

[1.584962500721156,
 0.14285714285714285,
 -0.8022139445529106,
 array([0.44651758, 0.46455345, 0.08892896])]